# Import Libraries and Construct Data

In [43]:
from __future__ import division
import pickle
import random
import numpy as np
from openrec.tf1.legacy import ImplicitModelTrainer
from openrec.tf1.legacy.utils import ImplicitDataset
from openrec.tf1.legacy.utils.evaluators import ImplicitEvalManager
from openrec.tf1.legacy.recommenders import CML, BPR
from openrec.tf1.legacy.utils.evaluators import AUC
from openrec.tf1.legacy.utils.samplers import PairwiseSampler

In [44]:
import os

SPLIT_UNBIASED_PREPROCESS = True

if SPLIT_UNBIASED_PREPROCESS:
    folder_name = "./Pre-Process-Split/"
else:
    folder_name = "./Post-Process-Split/"

seed = [76424236, 35427387, 90786253, 84636361, 61510520, 76632867, 27038789, 13359944, 28580255, 44670607]
split = 1

folder_name += f"/Split_data/split_{split}/"
if os.path.exists(folder_name) == False:
    os.makedirs(folder_name)

np.random.seed(seed=seed[split])

In [45]:
raw_data = dict()
raw_data['train_data'] = np.load(folder_name+"training_arr.npy")
raw_data['test_data_pos'] = np.load(folder_name+"unbiased-test_arr_pos.npy")
raw_data['test_data_neg'] = np.load(folder_name+"unbiased-test_arr_neg.npy")
raw_data['max_user'] = 15401
raw_data['max_item'] = 1001
batch_size = 8000
test_batch_size = 1000
display_itr = 1000

train_dataset = ImplicitDataset(raw_data['train_data'], raw_data['max_user'], raw_data['max_item'], name='Train')
test_dataset_pos = ImplicitDataset(raw_data['test_data_pos'], raw_data['max_user'], raw_data['max_item'])
test_dataset_neg = ImplicitDataset(raw_data['test_data_neg'], raw_data['max_user'], raw_data['max_item'])

# Define & Load Model

In [46]:
#Code to avoid tf using cached embeddings
import tensorflow as tf
tf.compat.v1.reset_default_graph()

In [47]:
cml_model = CML(batch_size=batch_size, max_user=train_dataset.max_user(), max_item=train_dataset.max_item(),
    dim_embed=50, l2_reg=0.001, opt='Adam', sess_config=None)
sampler = PairwiseSampler(batch_size=batch_size, dataset=train_dataset, num_process=4)
model_trainer = ImplicitModelTrainer(batch_size=batch_size, test_batch_size=test_batch_size,
                                     train_dataset=train_dataset, model=cml_model, sampler=sampler,
                                     eval_save_prefix=folder_name+"yahoo",
                                     item_serving_size=500)
auc_evaluator = AUC()

In [48]:
cml_model.load(folder_name+"cml-yahoo")

INFO:tensorflow:Restoring parameters from ./Pre-Process-Split/Split_data/split_4/cml-yahoo


# Generate Raw Results

In [49]:
model_trainer._eval_manager = ImplicitEvalManager(evaluators=[auc_evaluator])
model_trainer._num_negatives = 200
model_trainer._exclude_positives([train_dataset, test_dataset_pos, test_dataset_neg])
model_trainer._sample_negatives(seed=10)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-pos-unbiased"
model_trainer._evaluate_partial(test_dataset_pos)

model_trainer._eval_save_prefix = folder_name+"cml-yahoo-test-neg-unbiased"
model_trainer._evaluate_partial(test_dataset_neg)

[Subsampling negative items]


100%|██████████| 15400/15400 [02:48<00:00, 91.41it/s]


{'AUC': [0.4639,
  0.49104377104377106,
  0.4847333333333334,
  0.5047666666666667,
  0.5218833333333334,
  0.4772333333333334,
  0.5072500000000001,
  0.5188666666666667,
  0.5064166666666667,
  0.5050836120401337,
  0.4521666666666666,
  0.4632833333333334,
  0.49311666666666665,
  0.49200000000000005,
  0.47340000000000004,
  0.51645,
  0.49765000000000004,
  0.43474662162162164,
  0.4837,
  0.5196333333333333,
  0.5147666666666667,
  0.5275500000000001,
  0.46495000000000003,
  0.4783946488294315,
  0.5305166666666666,
  0.5344833333333333,
  0.48582214765100673,
  0.6222909698996656,
  0.5143000000000001,
  0.52755,
  0.5072742474916389,
  0.4664166666666667,
  0.51415,
  0.5129833333333332,
  0.5088999999999999,
  0.4808,
  0.5072333333333334,
  0.48816666666666675,
  0.48203333333333326,
  0.49793333333333334,
  0.52185,
  0.4855666666666667,
  0.51525,
  0.5615000000000001,
  0.5121333333333333,
  0.5015384615384615,
  0.48518333333333336,
  0.5028333333333334,
  0.539566666666

# Evaluation

In [50]:
def eq(infilename, infilename_neg, trainfilename, gamma=-1.0, K=1):
    infile = open(infilename, 'rb')
    infile_neg = open(infilename_neg, 'rb')
    P = pickle.load(infile)
    infile.close()
    P_neg = pickle.load(infile_neg)
    infile_neg.close()
    NUM_NEGATIVES = P["num_negatives"]
    #
    for theuser in P["users"]:
        neg_items = list(P_neg["user_items"][theuser][NUM_NEGATIVES:])
        neg_scores = list(P_neg["results"][theuser][NUM_NEGATIVES:])
        P["user_items"][theuser] = list(neg_items) + list(P["user_items"][theuser][NUM_NEGATIVES:])
        P["results"][theuser] = list(neg_scores) + list(P["results"][theuser][NUM_NEGATIVES:])
    #
    Zui = dict()
    Ni = dict()
    # fill in dictionary Ni
    trainset = np.load(trainfilename)
    for i in trainset['item_id']:
        if i in Ni:
            Ni[i] += 1
        else:
            Ni[i] = 1
    del trainset
    # count #users with non-zero item frequencies
    nonzero_user_count = 0
    for theuser in P["users"]:
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        for pos_item in pos_items:
            if pos_item in Ni:
                nonzero_user_count += 1
                break
    # fill in dictionary Zui
    for theuser in P["users"]:
        all_scores = np.array(P["results"][theuser])
        pos_items = P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]
        pos_scores = P["results"][theuser][len(P_neg["results"][theuser][NUM_NEGATIVES:]):]
        for i, pos_item in enumerate(pos_items):
            pos_score = pos_scores[i]
            Zui[(theuser, pos_item)] = float(np.sum(all_scores > pos_score))
    # calculate per-user scores
    sum_user_auc = 0.0
    sum_user_recall = 0.0
    for theuser in P["users"]:
        numerator_auc = 0.0
        numerator_recall = 0.0
        denominator = 0.0
        for theitem in P["user_items"][theuser][len(P_neg["user_items"][theuser][NUM_NEGATIVES:]):]:
            if theitem not in Ni:
                continue
            pui = np.power(Ni[theitem], (gamma + 1) / 2.0)
            numerator_auc += (1 - Zui[(theuser, theitem)] / len(P["user_items"][theuser])) / pui
            # Calcolo il Recall a 1, vedi nota 6 paper
            if Zui[(theuser, theitem)] < K:
                numerator_recall += 1.0 / pui
            denominator += 1 / pui
        if denominator > 0:
            sum_user_auc += numerator_auc / denominator
            sum_user_recall += numerator_recall / denominator

    return {
        "auc"       : sum_user_auc / nonzero_user_count,
        "recall"    : sum_user_recall / nonzero_user_count
    }

In [51]:
eq(folder_name+"cml-yahoo-test-pos-unbiased_evaluate_partial.pickle", folder_name+"cml-yahoo-test-neg-unbiased_evaluate_partial.pickle", folder_name+"training_arr.npy", gamma=1.5)

{'auc': 0.6623916333857374, 'recall': 0.010010216484624588}

In [56]:
results_cml = {
    0 : {'auc': 0.6633917641312187, 'recall': 0.01068920704608065},
    1 : {'auc': 0.6682286637673148, 'recall': 0.011689724800793281},
    2 : {'auc': 0.6731025043104407, 'recall': 0.008747459473265117},
    3: {'auc': 0.6679784500137645, 'recall': 0.009762657232517229},
    4: {'auc': 0.6623916333857374, 'recall': 0.010010216484624588},

}

In [57]:
auc_values = [results_cml[i]['auc'] for i in results_cml]
recall_values = [results_cml[i]['recall'] for i in results_cml]

average_auc = sum(auc_values) / len(auc_values)
average_recall = sum(recall_values) / len(recall_values)

average_auc, average_recall


(0.6670186031216951, 0.010179853007456174)